In [1]:
import matplotlib.pyplot as plt
import numpy as np
import json
import os
from scipy.io import loadmat
from pandas import DataFrame
import glob
import scipy.signal as spsig
from sklearn import preprocessing
import pandas as pd
import csv
from tqdm import tqdm

### testing feature extraction using one file

In [2]:
import eeglib

In [3]:
mat = loadmat('data/Pat1Train/Pat1Train_1_1.mat')
data = mat['data']
data

array([[ 19.369873 ,  -1.5776367, -41.794434 , ...,  62.809326 ,
         46.628662 , -18.094238 ],
       [ 23.331055 ,  -4.599121 , -36.759033 , ...,  70.79883  ,
         48.575684 , -17.087158 ],
       [ 25.345215 , -10.574463 , -34.744873 , ...,  84.831055 ,
         52.604004 , -17.087158 ],
       ...,
       [-15.610107 , -36.624756 ,  21.182617 , ..., -25.07666  ,
        -11.312988 ,  30.85083  ],
       [-24.60669  , -28.63501  ,  19.168457 , ..., -28.0979   ,
        -12.320068 ,  31.790771 ],
       [-34.610596 , -26.62085  ,  19.168457 , ..., -31.119385 ,
        -10.305908 ,  28.83667  ]], dtype=float32)

In [5]:
data.shape

(240000, 16)

In [4]:
# Perform z-score normalization - taken from eeglib helper function
from scipy.stats import zscore
data = zscore(data)
print(data.shape)
data

(240000, 16)


array([[ 0.5056598 , -0.05194973, -1.3151745 , ...,  1.2495348 ,
         1.0547212 , -0.56735015],
       [ 0.60906696, -0.14766373, -1.1573074 , ...,  1.4091601 ,
         1.0989348 , -0.53584874],
       [ 0.66164684, -0.33694947, -1.0941607 , ...,  1.6895155 ,
         1.1904111 , -0.53584874],
       ...,
       [-0.4074969 , -1.1621656 ,  0.65924656, ..., -0.5063733 ,
        -0.26103473,  0.9636486 ],
       [-0.64235383, -0.9090679 ,  0.59609973, ..., -0.5667358 ,
        -0.2839038 ,  0.9930499 ],
       [-0.903507  , -0.8452638 ,  0.59609973, ..., -0.6271032 ,
        -0.23816566,  0.90064585]], dtype=float32)

In [6]:
eeg_data = eeglib.eeg.EEG(240000, 400, 16)

In [7]:
eeg_data.set(data)
eeg_data.getChannel()

array([[ 0.5056598 ,  0.60906696,  0.66164684, ..., -0.4074969 ,
        -0.64235383, -0.903507  ],
       [-0.05194973, -0.14766373, -0.33694947, ..., -1.1621656 ,
        -0.9090679 , -0.8452638 ],
       [-1.3151745 , -1.1573074 , -1.0941607 , ...,  0.65924656,
         0.59609973,  0.59609973],
       ...,
       [ 1.2495348 ,  1.4091601 ,  1.6895155 , ..., -0.5063733 ,
        -0.5667358 , -0.6271032 ],
       [ 1.0547212 ,  1.0989348 ,  1.1904111 , ..., -0.26103473,
        -0.2839038 , -0.23816566],
       [-0.56735015, -0.53584874, -0.53584874, ...,  0.9636486 ,
         0.9930499 ,  0.90064585]], dtype=float32)

In [35]:
# Perform independent component analysis - taken from eeglib helper function
# from sklearn.decomposition import FastICA
# ica = FastICA()
# eeg_data = ica.fit_transform(np.transpose(data)).transpose()
# print(eeg_data.shape)
# eeg_data
# eeg_data.names = [str(i) for i in range(eeg_data.nChannels)]

In [44]:
cross_corr = eeg_data.CCC()
print(len(cross_corr))
cross_corr

120


[0.2075937210294459,
 0.025014084603957237,
 -0.021925934258798636,
 -0.03869724021221346,
 -0.08455400886142665,
 -0.29135599702171994,
 -0.14527300538784696,
 0.1842536133268646,
 0.06394971086252797,
 -0.19605134717638945,
 0.0014547716909196085,
 -0.1325385888760158,
 -0.025772017496249047,
 -0.2088674500836511,
 -0.2713144691992969,
 0.06321005873593635,
 0.05407806951027934,
 -0.027136577204112645,
 -0.08553582590869296,
 -0.19399376823334977,
 -0.09236643025228637,
 0.05471469083286263,
 -0.02392553855410736,
 -0.13181542248575448,
 -0.04871422112030129,
 -0.1363358825215131,
 -0.0662455295942018,
 -0.1778090240569538,
 -0.148408277498302,
 0.1026770262661103,
 -0.04228109318765608,
 -0.048022513056169926,
 -0.09856521051969651,
 -0.10320593811761036,
 -0.1207164990212245,
 -0.09330529201137026,
 -0.03515662812388502,
 -0.15648815236607747,
 -0.0592284235786844,
 -0.11577572245480616,
 -0.08182837694272094,
 0.007381414950422159,
 -0.008015711984088915,
 0.030613750665298596,
 -

In [46]:
# May want to extract further features from these
dft = eeg_data.DFT()
print(dft.shape)
dft

(16, 240000)


array([[-5.72200166e-04   +0.j        , -1.31280180e+03 +153.03920553j,
        -1.36268461e+03-1802.46715592j, ...,
         1.03379214e+03+1582.94622156j, -1.36268461e+03+1802.46715592j,
        -1.31280180e+03 -153.03920553j],
       [-8.33345111e-05   +0.j        , -5.70830721e+02 -365.77771653j,
        -7.76887036e+02 -310.75261103j, ...,
        -3.36130680e+03-2069.498746j  , -7.76887036e+02 +310.75261103j,
        -5.70830721e+02 +365.77771653j],
       [-1.93009386e-03   +0.j        , -1.70603999e+03-2165.18468209j,
        -2.18092861e+03 -584.09536729j, ...,
         1.13361403e+03-1199.50896981j, -2.18092861e+03 +584.09536729j,
        -1.70603999e+03+2165.18468209j],
       ...,
       [ 1.12876296e-06   +0.j        ,  2.40773153e+02 +154.75972788j,
        -5.25778351e+01 -421.22043301j, ...,
        -4.32772099e+02 +225.44442639j, -5.25778351e+01 +421.22043301j,
         2.40773153e+02 -154.75972788j],
       [-8.89197225e-04   +0.j        ,  2.32743860e+01 +304.0636094

In [47]:
psd = eeg_data.PSD()
print(psd.shape)
psd

(16, 60001)


array([[6.30201772e-03, 6.42774701e-02, 1.21216483e-01, ...,
        5.40983710e-05, 9.18860605e-05, 2.15717937e-05],
       [1.01254195e-01, 1.19713247e-01, 2.10533068e-01, ...,
        1.59437623e-05, 6.76702393e-06, 3.95068582e-06],
       [1.24585303e-02, 1.07872628e-01, 3.66541743e-01, ...,
        6.09537128e-05, 6.09651725e-05, 8.06941262e-06],
       ...,
       [8.69371230e-04, 3.75573500e-03, 3.30365566e-03, ...,
        6.46317421e-05, 8.17141045e-05, 1.25632387e-05],
       [9.89785069e-04, 1.03327155e-03, 2.22348329e-03, ...,
        3.24287830e-05, 4.07124862e-05, 9.65727395e-06],
       [4.36167009e-02, 5.03733866e-02, 7.85352066e-02, ...,
        3.47829700e-06, 7.43668306e-06, 5.67745701e-06]], dtype=float32)

In [49]:
b_pwr = eeg_data.bandPower()
print(len(b_pwr))
b_pwr

16


[{'delta': 3230.7743067885644,
  'theta': 2901.5455136654373,
  'alpha': 2227.2219208079036,
  'beta': 8524.262259497744},
 {'delta': 3329.9140890338813,
  'theta': 2451.29735777121,
  'alpha': 1677.90700295107,
  'beta': 6503.392037386898},
 {'delta': 3485.8018925333536,
  'theta': 2759.0576202237344,
  'alpha': 1559.1964969740643,
  'beta': 5709.1237298196775},
 {'delta': 4751.180822727143,
  'theta': 2247.576704107012,
  'alpha': 1067.7594346344424,
  'beta': 1914.17748008427},
 {'delta': 3978.4015567415136,
  'theta': 2547.626055095141,
  'alpha': 1687.04270113324,
  'beta': 4519.934439402942},
 {'delta': 4585.281421542576,
  'theta': 2984.715818590763,
  'alpha': 1691.6948216100914,
  'beta': 3273.8000407624027},
 {'delta': 5155.536431926746,
  'theta': 2405.0225361689786,
  'alpha': 1462.3729744468403,
  'beta': 2775.969187247783},
 {'delta': 4868.176126841488,
  'theta': 2908.340467679444,
  'alpha': 1778.8006345727017,
  'beta': 3295.8193010884215},
 {'delta': 3741.541603325256

In [51]:
pfd = eeg_data.PFD()
print(pfd.shape)
pfd

(16,)


array([1.00628385, 1.00736735, 1.00634725, 1.00711838, 1.00720861,
       1.00773353, 1.00732619, 1.0082046 , 1.00556184, 1.00762961,
       1.00709363, 1.00725356, 1.00704122, 1.00788659, 1.00751912,
       1.00777452])

In [52]:
# Takes nearly 7 min for one data set
hfd = eeg_data.HFD()
print(hfd.shape)
hfd

(16,)


array([2.08974159, 2.09005442, 2.08523611, 2.08536825, 2.09797317,
       2.09107063, 2.07873182, 2.08096211, 2.08113214, 2.08704601,
       2.09005161, 2.08264903, 2.0759342 , 2.08104698, 2.07799274,
       2.08577541])

In [53]:
h_act = eeg_data.hjorthActivity()
print(h_act.shape)
h_act

(16,)


array([1.0000002 , 1.0000001 , 1.0000001 , 0.9999999 , 1.0000001 ,
       1.0000001 , 0.9999999 , 0.99999994, 0.9999999 , 0.9999998 ,
       1.0000002 , 1.0000001 , 1.        , 0.99999994, 0.9999998 ,
       1.        ], dtype=float32)

In [54]:
h_mob = eeg_data.hjorthMobility()
print(h_mob.shape)
h_mob

(16,)


array([0.22832082, 0.20242207, 0.16912092, 0.07659574, 0.20078827,
       0.13147166, 0.09850346, 0.1563216 , 0.12837006, 0.14598298,
       0.12875019, 0.0928133 , 0.13755189, 0.145386  , 0.11366633,
       0.09203342], dtype=float32)

In [55]:
h_comp = eeg_data.hjorthComplexity()
print(h_comp.shape)
h_comp

(16,)


array([2.234388 , 2.8354225, 3.21677  , 6.5886946, 2.6191268, 4.321631 ,
       5.2256317, 4.490713 , 3.1314335, 4.269868 , 4.596317 , 6.8550835,
       4.9878626, 4.415425 , 5.500521 , 5.839792 ], dtype=float32)

In [9]:
# kills kernel
# synch = eeg_data.synchronizationLikelihood()
# print(synch.shape)
# synch

Error: Canceled future for execute_request message before replies were done

In [8]:
eng = eeg_data.engagementLevel()
print(eng.shape)
eng

()


0.8211801958860729

In [9]:
# Takes 15 minutes for one set
entr = eeg_data.sampEn()
print(entr.shape)
entr

(16,)


array([0.66478062, 0.69792145, 0.53560566, 0.28111452, 0.43470724,
       0.44678765, 0.34707467, 0.4331923 , 0.44776052, 0.47347458,
       0.36929238, 0.24718861, 0.38719925, 0.39869132, 0.3576908 ,
       0.32588023])

In [10]:
# Takes 5 minutes for one set
lzc = eeg_data.LZC()
print(lzc.shape)
lzc

(16,)


array([0.37845389, 0.36430469, 0.29467573, 0.18282257, 0.27315405,
       0.24775996, 0.23063198, 0.24269603, 0.26012189, 0.26943057,
       0.21938708, 0.220802  , 0.2532707 , 0.26391983, 0.23725976,
       0.20583364])

In [11]:
dfa = eeg_data.DFA()
print(dfa.shape)
dfa

(16,)


array([0.79179262, 0.90243462, 0.95069303, 0.94625632, 0.77714828,
       0.78422883, 0.76549872, 0.7912176 , 0.77884951, 0.79690753,
       0.8593628 , 0.75977751, 0.83875832, 0.69937404, 0.72577606,
       0.88119722])

In [13]:
# Took ~50 minutes
dtw = eeg_data.DTW()
print(len(dtw))
dtw

120


[153951.09141501517,
 173248.81788435695,
 151860.83103856328,
 177430.67191673082,
 141573.42405462964,
 185070.68611051887,
 148450.5555232237,
 176506.5515674064,
 144680.48193194374,
 138998.9140048524,
 160160.96529287793,
 149570.42054581334,
 150369.87462279474,
 166994.68194212357,
 153650.5798765932,
 131951.58342887246,
 130901.20295178425,
 138687.3929495269,
 138562.3472244736,
 139107.89751505095,
 156169.51665221888,
 147143.00875209004,
 156213.1399517859,
 148864.7457751506,
 155729.01216937904,
 162259.46454280894,
 148856.41007554543,
 143895.89681623538,
 148811.9416499702,
 135376.86079495627,
 135893.4883983928,
 146631.38863933051,
 140255.04442193318,
 144500.2660165123,
 146002.82784556953,
 140343.61558617215,
 145352.12428826746,
 152062.66376186535,
 143775.36055799056,
 152757.06230457558,
 156973.38838768733,
 132647.8534833067,
 123548.36105431837,
 126872.02738647675,
 130656.08566381293,
 132848.97471695242,
 143774.98454634426,
 127478.98459065228,
 134

### preprocessing data

In [2]:
read_dir = 'data/Pat1Train'
filenames = sorted(os.listdir(read_dir))
train_filenames = []
for filename in filenames:
    train_filenames.append(filename)
n = len(train_filenames)
image = loadmat(read_dir + '/' + train_filenames[0], squeeze_me=True)
X = np.zeros(((n,) + image['data'].shape), dtype='float32')
y = np.zeros(n, dtype='int8')

filename_to_idx = {}
for i, filename in enumerate(train_filenames):
    data= loadmat(read_dir + '/' + filename, squeeze_me=True)
    X[i] = data['data']
    y[i] = 1 if filename.endswith('_1.mat') else 0
    filename_to_idx['Pat1' + '/' + filename] = i

In [3]:
print(X.shape)
print(y.shape)
unique, counts = np.unique(y, return_counts=True)
dict(zip(unique, counts))

(826, 240000, 16)
(826,)


{0: 570, 1: 256}

In [ ]:
pat = 1
outfile = 'pat_'+str(pat)+'_feature_train.csv'

label = y

mydata = []
bands=[0.1,4,8,12,30,70,180]
rate = 400.
for i, x in tqdm(enumerate(X)):
        data_full = x.T
        output = []
        featureList = []
        featureList.append('Label')
        output.append(y[i])
        featureList.append('Pat')
        output.append(pat)
        data_full = data_full.astype(float)
        data = preprocessing.scale(data_full, axis=1,with_std=True)

        # get correlation Coef. this will be 16x16
        h=np.corrcoef(data)
        h=np.nan_to_num(h)
        # only want upper triangle
        ind = np.triu_indices(16, 1)
        htri = h[ind]
        for ii in range(np.size(htri)):
                featureList.append('coef%i'%(ii))
                output.append(htri[ii])


        # for j in range(19):
        #         data = data_full[:,j*int(rate*60/2):(j)*int(rate*60/2) + int(rate)*60]
        #         data = preprocessing.scale(data, axis=1,with_std=True)
                
        #         for k in range(16):
        #                 hold = data[k,:]
        #                 f,psd = spsig.welch(hold, fs=400, nperseg=2000)
        #                 psd = np.nan_to_num(psd)
        #                 psd /= psd.sum()
        #                 for c in range(1,len(bands)):
        #                         featureList.append('BandEnergy_%i_%i_%i'%(j,k,c))
        #                         output.append(psd[(f>bands[c-1])&(f<bands[c])].sum())


        mydata.append(pd.DataFrame({'Features':output},index=featureList).T)

trainSample = pd.concat(mydata,ignore_index=True)
trainSample.to_csv(outfile)